Lab 3

Task:
1. Write a program for planning a path from the initial point to the final point bypassing obstacles using the potential field method.
2. Independently determine the initial point of movement, the final point, and obstacles in the workspace.
3. The program code must reflect all the steps for constructing the potential field of the workspace, the order of calculating the gradient, and determining the subsequent direction of movement.
4. Illustrate achieved results.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import roboticstoolbox as rtb
from spatialmath import SE3
from math import pi

1. Loading the manipulator model Stanford Arm


In [4]:
L = [
    rtb.RevoluteMDH(d=0.412, qlim=[-pi, pi]),
    rtb.RevoluteMDH(d=0.154, alpha=-pi/2, qlim=[-pi/2, pi/2]),
    rtb.PrismaticMDH(alpha=pi/2, qlim=[0.2032, 0.9144]),
    rtb.RevoluteMDH(qlim=[-pi, pi]),
    rtb.RevoluteMDH(alpha=-pi/2, qlim=[-pi, pi]),
    rtb.RevoluteMDH(d=0.263, alpha=pi/2, qlim=[-pi, pi])
]

robot = rtb.DHRobot(L, name='Stanford Arm')
print(robot)


DHRobot: Stanford Arm, 6 joints (RRPRRR), dynamics, modified DH parameters
┌──────┬────────┬──────┬───────┬─────────┬────────┐
│ aⱼ₋₁ │  ⍺ⱼ₋₁  │  θⱼ  │  dⱼ   │   q⁻    │   q⁺   │
├──────┼────────┼──────┼───────┼─────────┼────────┤
│  0.0 │   0.0° │   q1 │ 0.412 │ -180.0° │ 180.0° │
│  0.0 │ -90.0° │   q2 │ 0.154 │  -90.0° │  90.0° │
│  0.0 │  90.0° │ 0.0° │    q3 │  0.2032 │ 0.9144 │
│  0.0 │   0.0° │   q4 │   0.0 │ -180.0° │ 180.0° │
│  0.0 │ -90.0° │   q5 │   0.0 │ -180.0° │ 180.0° │
│  0.0 │  90.0° │   q6 │ 0.263 │ -180.0° │ 180.0° │
└──────┴────────┴──────┴───────┴─────────┴────────┘

┌──┬──┐
└──┴──┘



In [19]:
m = [3.2, 2.8, 4.5, 1.2, 1.0, 0.8]  # Для 6 звеньев
r = [
    [0, 0, 0.15],    # Звено 1 (база)
    [0.1, 0, 0.2],   # Звено 2 
    [0, 0, 0.3],     # Звено 3 (призматическое)
    [0.05, 0, 0.1],  # Звено 4
    [0, 0.03, 0.08], # Звено 5
    [0, 0, 0.05]     # Звено 6 (эффектор)
]
I = [
    [0.12, 0.10, 0.08, 0, 0, 0],    # Звено 1
    [0.25, 0.18, 0.15, 0, 0, 0.02], # Звено 2
    [0.35, 0.30, 0.40, 0, 0, 0],    # Звено 3
    [0.02, 0.03, 0.01, 0, 0, 0],    # Звено 4
    [0.015, 0.02, 0.01, 0, 0, 0],   # Звено 5
    [0.005, 0.004, 0.003, 0, 0, 0]  # Звено 6
]
Jm = [0.005, 0.004, 0.006, 0.001, 0.0008, 0.0005]
B = [0.15, 0.12, 0.18, 0.08, 0.06, 0.04]

Tc = [
    [0.25, -0.22],  # Звено 1 
    [0.20, -0.18],  # Звено 2
    [0.30, -0.28],  # Звено 3
    [0.15, -0.13],  # Звено 4
    [0.12, -0.10],  # Звено 5
    [0.08, -0.07]   # Звено 6
]

G = [120.0, 100.0, 80.0, 50.0, 40.0, 30.0]

qlim = [
    [-pi, pi],       # Звено 1 (револьверное)
    [-pi/2, pi/2],   # Звено 2
    [0.0, 0.9144],   # Звено 3 (призматическое, 0-36")
    [-2*pi, 2*pi],   # Звено 4
    [-pi, pi],       # Звено 5
    [-pi, pi]        # Звено 6
]


In [21]:
from roboticstoolbox import DHRobot, RevoluteDH, PrismaticDH

L = [
    RevoluteDH(d=0.412, m=m[0], r=r[0], I=I[0], Jm=Jm[0], B=B[0], Tc=Tc[0], G=G[0], qlim=qlim[0]),
    RevoluteDH(d=0.154, alpha=-pi/2, m=m[1], r=r[1], I=I[1], Jm=Jm[1], B=B[1], Tc=Tc[1], G=G[1], qlim=qlim[1]),
    PrismaticDH(alpha=pi/2, qlim=qlim[2], m=m[2], r=r[2], I=I[2], Jm=Jm[2], B=B[2], Tc=Tc[2], G=G[2]),
    RevoluteDH(m=m[3], r=r[3], I=I[3], Jm=Jm[3], B=B[3], Tc=Tc[3], G=G[3], qlim=qlim[3]),
    RevoluteDH(alpha=-pi/2, m=m[4], r=r[4], I=I[4], Jm=Jm[4], B=B[4], Tc=Tc[4], G=G[4], qlim=qlim[4]),
    RevoluteDH(d=0.263, alpha=pi/2, m=m[5], r=r[5], I=I[5], Jm=Jm[5], B=B[5], Tc=Tc[5], G=G[5], qlim=qlim[5])
]

robot = DHRobot(L, name='Stanford Arm')

In [22]:
print(robot.links[0].dyn())

m     =       3.2 
r     =         0        0     0.15 
        |     0.12        0        0 | 
I     = |        0      0.1        0 | 
        |        0        0     0.08 | 
Jm    =     0.005 
B     =      0.15 
Tc    =      0.25(+)    -0.22(-) 
G     =   1.2e+02 
qlim  =      -3.1 to      3.1


2. Definition of working poses 
Initial pose (q = [θ1, θ2, d3, θ4, θ5, θ6])

In [5]:
q_start = np.array([0, -np.pi/2, 0.5, 0, np.pi/2, 0])

The target pose of the effector

In [6]:
goal_pose = SE3(0.8, 0.2, 0.5)  # x, y, z в метрах
q_goal = robot.ikine_LM(goal_pose).q  # Решение обратной кинематики

3. Identify obstacles

In [7]:
obstacles = [
    {'center': np.array([0.4, 0.0, 0.3]), 'radius': 0.15},
    {'center': np.array([0.6, 0.2, 0.4]), 'radius': 0.1}
]

4. Potential Functions for Stanford Arm

In [8]:
def attractive_potential(q):
    """Притяжение к целевой позе эффектора"""
    current_pose = robot.fkine(q).t
    return 0.5 * np.linalg.norm(goal_pose.t - current_pose)**2

In [14]:
def repulsive_potential(q):
    """Отталкивание от препятствий для всех звеньев"""
    rep = 0
    
    # Получаем позиции всех звеньев для полной конфигурации q
    poses = robot.fkine_all(q).data  # Все позы звеньев
    
    for i in range(robot.n):
        # Позиция i-го звена (используем полную конфигурацию)
        link_pose = poses[i+1].t  # poses[0] - base, poses[1] - first link
        
        for obs in obstacles:
            distance = np.linalg.norm(link_pose - obs['center']) - obs['radius']
            if distance < 0.2:  # Радиус влияния
                rep += 0.5 * (1/(distance + 1e-6) - 1/0.2)**2
    return rep * 50

5. Numerical calculation of the gradient

In [16]:
def compute_gradient(q, epsilon=1e-4):
    grad = np.zeros_like(q)
    for i in range(len(q)):
        q_plus = q.copy()
        q_plus[i] += epsilon
        q_minus = q.copy()
        q_minus[i] -= epsilon
        
        # Проверка ограничений суставов
        q_plus = np.clip(q_plus, robot.qlim[:,0], robot.qlim[:,1])
        q_minus = np.clip(q_minus, robot.qlim[:,0], robot.qlim[:,1])
        
        grad[i] = (attractive_potential(q_plus) + repulsive_potential(q_plus) 
                  - attractive_potential(q_minus) - repulsive_potential(q_minus)) / (2*epsilon)
    return grad


6. Trajectory planning

In [17]:
path = [q_start.copy()]
current_q = q_start.copy()
alpha = 0.05  # Скорость обучения
max_iter = 200

for _ in range(max_iter):
    grad = compute_gradient(current_q)
    current_q -= alpha * grad
    
    # Применение ограничений суставов
    current_q = np.clip(current_q, robot.qlim[:,0], robot.qlim[:,1])
    
    path.append(current_q.copy())
    
    # Проверка достижения цели
    if np.linalg.norm(robot.fkine(current_q).t - goal_pose.t) < 0.02:
        break

path = np.array(path)

ValueError: operands could not be broadcast together with shapes (6,) (2,) (2,) 

7. Visualization

In [ ]:
# Анимация движения
robot.plot(path, backend='swift', dt=0.05)

# График траектории в пространстве
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111, projection='3d')

# Траектория эффектора
ee_path = np.array([robot.fkine(q).t for q in path])
ax.plot(ee_path[:,0], ee_path[:,1], ee_path[:,2], 'b-', lw=2)

# Препятствия
for obs in obstacles:
    u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
    x = obs['center'][0] + obs['radius']*np.cos(u)*np.sin(v)
    y = obs['center'][1] + obs['radius']*np.sin(u)*np.sin(v)
    z = obs['center'][2] + obs['radius']*np.cos(v)
    ax.plot_surface(x, y, z, color='r', alpha=0.3)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.title('Траектория Stanford Arm с учётом препятствий')
plt.show()